# Environmental Solutions Lab: Adaptive Infrastructure
## Tasks IV & V: Predicting blackout events using machine learning
Intelligent Earth Center for Doctoral Training <br>
Friday 13th December 2024 <br>

Instructors:
* <alison.peard@ouce.ox.ac.uk><br>
* <yu.mo@chch.ox.ac.uk>


### Overview
In this task, you will use machine learning models to predict blackout events based on storm characteristics, blackout data, and socio-economic factors.

When we finish the tutorial, play around with the model. Maybe a different model will be better than XGBoost, or we have missed some useful predictors? Are there any other datasets we can use?

#### Environment set up

To begin, let's load all the packages we will need.

We use the `scikit-learn` library, which provides many generic machine learning methods like grid searching and train-test splitting. We want to use an `XGboost` model, so we need to also import the `xgboost` library.

We define some arguments at the start of our code: `SEED`, to ensure reproducibility and `NORMALISE` to select whether to normalise the data before feeding it into the model. Normalisation is not needed for tree-based methods like XGBoost, so we can leave this switched off for now.

We will be predicting a binary outcome, so we instantiate the classified model from `xgboost`, `XGBClassifier`.

We also set up a variable pointing to out data directory `WD`.

In [ ]:
import os
import shap
import numpy as np
import pandas as pd
import xgboost as xgb
import multiprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pprint import pp as prettyprint

SEED = 42
NORMALISE = False
XGBModel = xgb.XGBClassifier #  xgb.XGBRegressor for regression

WD = os.path.join(os.getcwd(), "data")

### Load and process the data

We will use the data we prepared in Tasks I-III. This should consist of clean storm characteristics from IBTrACS, blackout durations from the Blackout dataset, plus additional socioeconomic data. The pre-prepared file `blackout_with_socio.csv` provided, so we can just load this in.

We want to predict electricity outages (derived from blackout data) from storm characteristics and socioeconomic factors.

Begin by loading and visualising the data.

In [ ]:
df = pd.read_csv(os.path.join(WD, "blackout_with_socio.csv"))
print(f"Data has {len(df.columns)} columns and {len(df)} rows.")
df.head()

There are 30 columns. The `duration` column represents blackout duration and the remaining 29 are predictors. All of these are numeric except for `income`, which has string categories, which `XGBoost` cannot automatically handle. We need to encode this in a numeric format.

Two common techniques are _label encoding_ and _one-hot encoding_. Label encoding should only be used when there is a clear hierarchy between categories, e.g., high, medium, low. It assigns an ordinal value to each category corresponding to the hierarchy. One-hot encoding creates a binary column for each category.

Since country income has a clear hierarchy, low, lower-middle, upper-middle, and high, we opt for label-encoding.

In [ ]:
# categorical to ordinal
income_factors = {'L': 0, 'LM': 1, 'UM': 2, 'H': 3}
df['income'] = df['income'].map(income_factors)

The blackout duration variable takes integer values corresponding to the number of days of blackout experienced after an IBTrACs storm. This is a difficult prediction task. One way to simplify the problem, while still gaining useful information, is to set a value of interest and focus on predicting whether the blackout duration exceeds this value. Here, we will use the median duration.

In [ ]:
# convert duration to binary variable
median = df['duration'].median()
df['duration'] = (df['duration'] > median).astype(int)

Now, `duration` is a binary variable indicating whether the median blackout duration has been exceeded for each sample.

With the data processing has been done, split the dataset into train and test sets. We choose a subset of the possible predictor values, the maximum wind speed, the storm speed, the distance from the pixel to the storm track, and the socioeconomic income at that location. (Note we use the `SEED` variable to ensure reproduciblilty.)

In [ ]:
# train-test split
response = "duration"
regressors = ["windmax", "speed", "distToTrack", "income"]
df = df.loc[:, regressors + [response]].copy()

# 90:10 split
df = df[regressors + [response]]
train, test = train_test_split(df, train_size=0.9, shuffle=True, random_state=SEED)
train.head()

### XGBoost

XGBoost ([eXtreme Gradient Boosting](https://xgboost.readthedocs.io/en/stable/)) is an ensemble algorithm based-on the predictions of hundreds of decision trees.

There are two main types of ensemble algorithm:

* **bagging algorithms:** (or bootstrap aggregating) are based on drawing bootstrap samples from your training data. Random forests are a special case of bagging algorithm where the predictive features at each data split are also sub-sampled.

* **boosting algorithms:** are iterative approaches, where an ensemble consists of a sequence of weak learners, each tuned to compensate for the weaknesses in their predecessors. XGBoost is a gradient boosting algorithm where the weak learners are hundreds or thousands of decision trees.

XGBoost models have a large number of parameters relating to the base learners (decision trees) and the booster. These are described in detail in the [XGboost documentation](https://xgboost.readthedocs.io/en/stable/parameter.html).

We will use `scikit-learn`'s `GridSearchCV` to do cross-validatory gridsearch through a subset of these.

In [ ]:
# help(XGBModel)

In [ ]:
# define the model
X, y = train[regressors], train[response]

model = XGBModel(
    n_jobs=multiprocessing.cpu_count() // 2,
    tree_method="hist",
    objective="binary:logistic"
)

In [ ]:
# define the grid search
cv = GridSearchCV(
    model,
    {   "booster": ["gbtree"],            # default, could try "gblinear"
        "learning_rate": [0.1, 1],        # alias eta ~[0.01 - 0.3], lower leads to slower computation
        "min_split_loss": [1],            # alias gamma, regulation. ~[0,inf]. Start with 0 and check CV error rate. If you see train error >>> test error, bring gamma into action. Higher the gamma, lower the difference in train and test CV.
        "alpha": [1],                     # L1 regularization (equivalent to Lasso regression) on weights.
        "lambda": [1],                    # L2 regularization (equivalent to Ridge regression) on weights. It is used to avoid overfitting
        "max_depth": [8],                 # the depth of the tree.Larger the depth, more complex the model; higher chances of overfitting
        "min_child_weight": [6],
        "subsample": [0.8, 0.9],          # number of samples (observations) supplied to a tree. ~ (0.5-0.8)
        "colsample_bytree": [0.8, 0.9],   # the number of features (variables) supplied to a tree. [0,1] ~(0.5,0.9)
        "colsample_bylevel": [0.8, 0.9],  # the number of features (variables) supplied to each level. [0,1] ~(0.5,0.9)
        "colsample_bynode": [0.8, 0.9]    # the number of features (variables) supplied to each node. [0,1] ~(0.5,0.9)
        },
    verbose=1,
    n_jobs=multiprocessing.cpu_count() // 2,
)

In [ ]:
# perform the cv
cv.fit(X, y)

prettyprint(cv.best_score_)
prettyprint(cv.best_params_)

View the full CV results dataframe.

In [ ]:
# for col in cv_df.columns:print(col) # to see what columns are available

We can view the gridsearch results as a DataFrame. 

In [ ]:
cv_df = pd.DataFrame(cv.cv_results_)
param_cols = [col for col in cv_df.columns if col.startswith("param_")]
cv_df[["rank_test_score", "mean_test_score", "std_test_score"] + param_cols].sort_values(by="rank_test_score", ascending=True).head()

In [ ]:
# save parameters to a csv file 
best_params = pd.DataFrame.from_dict(cv.best_params_, orient="index")
best_params.columns = ["value"]
best_params.to_csv(os.path.join(WD, "best_params.csv"))

### Fit the best model

Now that we have finished hyperparameter tuning, we can fit the model with the best parameters.

In [ ]:
best_params = pd.read_csv(os.path.join(WD, "best_params.csv"), index_col=0)
best_params = best_params.to_dict()["value"]

In [ ]:
best_model = XGBModel(
    n_jobs=multiprocessing.cpu_count() // 2,
    tree_method="hist",
    **best_params
)
best_model.fit(X, y)

### Look at results

Get fits and predictions by applying the fitted XGBoost model to the train and test data.

In [ ]:
y_train = train[response]
y_test = test[response]
y_fit = best_model.predict(train[regressors])
y_pred = best_model.predict(test[regressors])

Since this is a binary problem, accuracy alone does not give us all the information. We calculate the confusion (contingency) matrix of true positives, true negatives, false positives, and false negatives and look at some popular derived metrics.

* accuracy: overall proportion correct
* bias: balance between under- and over-prediction (1 indicates unbiased)
* precision / specificity: are we avoiding all the negatives?
* recall / sensitivity: are we catching all the positives?
* CSI balances precision and recall

In [ ]:
# classification metrics
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Observation'], colnames=['Prediction'])
a = confusion_matrix[1][1] # true positives
b = confusion_matrix[0][1] # false positives
c = confusion_matrix[1][0] # false negatives
d = confusion_matrix[0][0] # true negatives

accuracy = np.mean(y_test == y_pred)

bias = (a + b) / (a + c)
precision = a / (a + b)
recall = a / (a + c)
csi = a / (a + b + c) # critical success index (f2)


print('Results:\n--------')
print(f'Accuracy: {accuracy:.4f}')
print(f'Bias: {bias:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'CSI: {csi:.2f}')

### Feature importance and SHAP

XGBoost's built-in feature importance is based-on summing up the feature importances for all the decision trees in the ensemble.

In [ ]:
# look at built-in feature importances (not recommended)
rank = best_model.feature_importances_.argsort()
plt.barh(df.columns[rank], best_model.feature_importances_[rank])
plt.xlabel('Default XGBoost feature importances')

### Afternoon assignment

For the rest of the tutorial, split into  groups of 3--4 to explore the model, predictors, and hyperparameters more. At the end of the tutorial we will ask each group to present three slides and share what you've found.

#### Three slides

1.	Hyperparameter importance: identify the most relevant XGBoost hyperparameters for model tuning.
2.	Predictor importance: identify the most important predictors. Compare different variable importance metrics. Summarise key insights, focusing on the storm characteristics and socio-economic factors that most influence power outages. Discuss any patterns or trends identified in the data. 
3.	Report your best scores(s). Try other models/parameters/hyperparameters. Feel free to go more complex or more simple.


### (Extra) Shapely values with SHAP

SHAP is now widely used to determine feature importance and can provide a more information about how models make predictions. SHAP values are model-agnostic and generally reliable. Roughly, they calculate the expected marginal contribution of each variable across all samples. 

We use the python `shap` library. You can read the docs [here](https://shap.readthedocs.io/en/latest/).

In [ ]:
shap.initjs() # for plotting

explainer = shap.Explainer(best_model)
shap_values = explainer(train[regressors])

We can get a plot similar to the feature importance plot by plotting the average SHAP value across all samples. The results are similar to those of XGBoost's feature importance, except speed and windmax are swapped.

In [ ]:
shap.plots.bar(shap_values)

The $x$-axis shows the average absolute contribution it made to the outcome variable, in this case the log-odds of a blackout exceeding the median duration.

We can also look at the SHAP values for individual observations using a waterfall plot.

In [ ]:
# waterfall
shap.plots.waterfall(shap_values[0])

For sample zero, the maximum wind speed reduced the probability of a long blackout.

We can also view this as a force plot, which is basically a condensed waterfall plot.

In [ ]:
# force plot
shap.plots.force(shap_values[0])

A stacked force plot allows us to visualise the SHAP values of multiple variables at once. This is made by rotating the force plots to vertical for each observation, and interpolating between them to create smooth lines. Note the plot is interactive.

In [ ]:
# stacked force plot
shap.plots.force(shap_values[0:100])

Beeswarm plot provide an information-dense summary of how the top features impact the model's output. The $x$-axis position indicates the SHAP value of a predictor for each sample, while the shading indicates whether the predictor had a low or high value for that sample. 

In [ ]:
# beeswarm
shap.plots.beeswarm(shap_values)

We can also plot the relationship between the SHAP values and one or more predictors.

In [ ]:
shap.plots.scatter(shap_values[:, "windmax"], color=shap_values[:, "speed"])

#### References
1. Stephens (2013) Problems with binary pattern measures for flood model evaluation
2. https://shap.readthedocs.io/en/latest/
3. https://xgboost.readthedocs.io/en/stable/python/